<a href="https://colab.research.google.com/github/rachanabn20/MUCS-RANLP2025/blob/main/PolyHope_M_at_RANLP2025ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***PolyHope-M at RANLP2025***

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install num2words
!pip install emoji
!pip install contractions
!pip install emot
!pip install demoji

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.5/163.5 kB 5.8 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=2022f6932ea39fb2c92d54b9b9f7ca9bef4427353393fd64e94d0dfc020e4ab5
  Stored in directory: /root/.cache/pip/wheels/1a/b0/8c/4b75c4116c31f83c8f9f047231251e13cc74481cca4a78a9ce
Successfully built docopt
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.3/118.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 1.6 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
import string
import contractions
import re
import regex
import demoji
import emoji
import os
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.metrics import f1_score
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.tokenize import word_tokenize
from num2words import num2words
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix



In [5]:
# Download necessary NLTK data files
nltk.download('wordnet')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('vader_lexicon')
#nltk.download('whitespace')

# Define stopwords for each language
stopwords_en = set(stopwords.words('english'))
stopwords_de = set(stopwords.words('german'))
stopwords_es = set(stopwords.words('spanish'))

# Manually define Urdu stop words
stopwords_ur = set([
    "اور", "کا", "کی", "کے", "سے", "میں", "تو", "پر", "نے", "یہ", "ہے", "ہی", "کو", "نہیں",
    "اس", "کر", "لیکن", "کرنا", "کرنے", "کرتے", "کیا", "ہو", "جا", "سکتے", "وہ", "ہوتا",
    "ہوتی", "ہوتے", "تھا", "ہوئے", "بھی", "کسی", "کبھی", "کہاں", "کیوں", "کچھ",
    "لیا", "دیا", "دیے", "دیکھا", "گیا", "جب", "تب", "اگر", "مگر", "یہاں", "وہاں", "کہ",
    "کہا", "ساتھ", "بار", "دوسرے", "پہلے", "بعد", "چاہیے", "جیسے", "ایسے", "تیسرے",
    "چوتھے", "پانچویں", "چھٹے", "ساتویں", "آٹھویں", "نویں", "دسویں", "گیارہویں", "بارہویں"
])

# Combine all stopwords for the remove_stopwords function
all_stopwords = stopwords_en.union(stopwords_de, stopwords_es, stopwords_ur)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [6]:
# English train and dev datasets
en_train = pd.read_csv('/content/drive/MyDrive/Training_Data_PolyHope-M_2025/Training Data/en_train.csv')
en_dev = pd.read_csv('/content/drive/MyDrive/Training_Data_PolyHope-M_2025/Training Data/en_dev.csv')

# Spanish train and dev datasets
es_train = pd.read_csv('/content/drive/MyDrive/Training_Data_PolyHope-M_2025/Training Data/es_train.csv')
es_dev= pd.read_csv('/content/drive/MyDrive/Training_Data_PolyHope-M_2025/Training Data/es_dev.csv')

#Urdu train and dev datasets
ur_train = pd.read_csv('/content/drive/MyDrive/Training_Data_PolyHope-M_2025/Training Data/Ur_train.csv')
ur_dev= pd.read_csv('/content/drive/MyDrive/Training_Data_PolyHope-M_2025/Training Data/Ur_dev.csv')

#German train and dev datasets
de_train = pd.read_csv('/content/drive/MyDrive/Training_Data_PolyHope-M_2025/Training Data/German_train.csv')
de_dev= pd.read_csv('/content/drive/MyDrive/Training_Data_PolyHope-M_2025/Training Data/German_dev.csv')

# Test sets
en_test=pd.read_csv('/content/drive/MyDrive/TestWithNoLabel/TestWithNoLabel/en_test_without_labels.csv')
es_test=pd.read_csv('/content/drive/MyDrive/TestWithNoLabel/TestWithNoLabel/es_test_without_labels.csv')
ur_test=pd.read_csv('/content/drive/MyDrive/TestWithNoLabel/TestWithNoLabel/Ur_test_without_labels.csv')
de_test=pd.read_csv('/content/drive/MyDrive/TestWithNoLabel/TestWithNoLabel/German_test_without_labels.csv')

In [7]:
merge_train = pd.concat([en_train, es_train,ur_train,de_train], ignore_index=True)


In [8]:
# Text preprocessing
def expand_contractions(text):
    return contractions.fix(text)

def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

def remove_stopwords(text, language='english'):
    stopwords_set = {
        'english': stopwords_en,
        'german': stopwords_de,
        'spanish': stopwords_es,
        'urdu': stopwords_ur
    }
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stopwords_set[language]]
    return ' '.join(filtered_words)

def lemmatize_text(text, language='english'):
    if language == 'english':
        lemmatizer = WordNetLemmatizer()
        words = word_tokenize(text)
        lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
        return ' '.join(lemmatized_words)
    else:
        # For other languages, return the text as is
        return text

def stem_text(text, language='english'):
    stemmer = SnowballStemmer(language)
    words = word_tokenize(text)
    stemmed_words = [stemmer.stem(word) for word in words]
    return ' '.join(stemmed_words)

def convert_numbers_to_words(text, language='en'):
    superscript_map = str.maketrans('⁰¹²³⁴⁵⁶⁷⁸⁹', '0123456789')
    text = text.translate(superscript_map)
    tokens = word_tokenize(text)
    converted_tokens = []
    for token in tokens:
        if token.isdigit():
            try:
                converted_tokens.append(num2words(int(token), lang=language))
            except NotImplementedError:
                converted_tokens.append(token)
        else:
            converted_tokens.append(token)
    return ' '.join(converted_tokens)

def remove_non_ascii(text):
    return ''.join(char for char in text if ord(char) < 128)

def remove_tags(text):
    return re.sub(r'#\w+', '', text)

def preprocess_text(text, language='english'):
    text = text.lower()
    text = remove_non_ascii(text)
    text = expand_contractions(text)
    text = remove_tags(text)
    text = remove_punctuation(text)
    text = convert_numbers_to_words(text, language)
    text = remove_stopwords(text, language)
    text = lemmatize_text(text, language)
    text = stem_text(text, language)
    return text



In [9]:
# Preprocess the text data
merge_train['processed_text'] = merge_train['text'].apply(lambda x: preprocess_text(x))
en_dev['processed_text'] = en_dev['text'].apply(lambda x: preprocess_text(x))
es_dev['processed_text'] = es_dev['text'].apply(lambda x: preprocess_text(x))
ur_dev['processed_text'] = ur_dev['text'].apply(lambda x: preprocess_text(x))
de_dev['processed_text'] = de_dev['text'].apply(lambda x: preprocess_text(x))
en_test['processed_text'] = en_test['text'].apply(lambda x: preprocess_text(x))
es_test['processed_text'] = es_test['text'].apply(lambda x: preprocess_text(x))
ur_test['processed_text'] = ur_test['text'].apply(lambda x: preprocess_text(x))
de_test['processed_text'] = de_test['text'].apply(lambda x: preprocess_text(x))


# **Binary Classification**

**Dev Set**

In [10]:

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000)),
    ('clf', LogisticRegression(max_iter=1000))
])


pipeline.fit(merge_train['processed_text'], merge_train['binary'])


y_pred_es = pipeline.predict(es_dev['processed_text'])
y_pred_en = pipeline.predict(en_dev['processed_text'])
y_pred_ur = pipeline.predict(ur_dev['processed_text'])
y_pred_de = pipeline.predict(de_dev['processed_text'])


In [11]:
from sklearn.metrics import classification_report

print("Classification Report for Spanish (es):")
print(classification_report(es_dev['binary'], y_pred_es))

print("Classification Report for English (en):")
print(classification_report(en_dev['binary'], y_pred_en))

print("Classification Report for Urdu (ur):")
print(classification_report(ur_dev['binary'], y_pred_ur))

print("Classification Report for German (de):")
print(classification_report(de_dev['binary'], y_pred_de))

Classification Report for Spanish (es):
              precision    recall  f1-score   support

        Hope       0.77      0.75      0.76      1879
    Not Hope       0.77      0.78      0.78      1958

    accuracy                           0.77      3837
   macro avg       0.77      0.77      0.77      3837
weighted avg       0.77      0.77      0.77      3837

Classification Report for English (en):
              precision    recall  f1-score   support

        Hope       0.77      0.79      0.78       834
    Not Hope       0.78      0.75      0.76       816

    accuracy                           0.77      1650
   macro avg       0.77      0.77      0.77      1650
weighted avg       0.77      0.77      0.77      1650

Classification Report for Urdu (ur):
              precision    recall  f1-score   support

        Hope       0.75      0.93      0.83       794
    Not Hope       0.92      0.73      0.81       884

    accuracy                           0.82      1678
   macro av

**Test set**

In [12]:
y_pred_es = pipeline.predict(es_test['processed_text'])
y_pred_en = pipeline.predict(en_test['processed_text'])
y_pred_ur = pipeline.predict(ur_test['processed_text'])
y_pred_de = pipeline.predict(de_test['processed_text'])

In [13]:
#submission file

y_pred_es = pd.DataFrame(data=y_pred_es, columns=['final_prediction'])
Submisssion_es = pd.DataFrame()
Submisssion_es['Text'] = es_test['text']
Submisssion_es['Tag'] = y_pred_es
Submisssion_es.to_csv('predictions.csv', index = None)

In [14]:
#submission file

y_pred_en = pd.DataFrame(data=y_pred_en, columns=['final_prediction'])
Submisssion_en = pd.DataFrame()
Submisssion_en['Text'] = en_test['text']
Submisssion_en['Tag'] = y_pred_en
Submisssion_en.to_csv('predictions.csv', index = None)

In [15]:
#submission file

y_pred_ur = pd.DataFrame(data=y_pred_ur, columns=['final_prediction'])
Submisssion_ur = pd.DataFrame()
Submisssion_ur['Text'] = ur_test['text']
Submisssion_ur['Tag'] = y_pred_ur
Submisssion_ur.to_csv('predictions.csv', index = None)

In [16]:
#submission file

y_pred_de = pd.DataFrame(data=y_pred_de, columns=['final_prediction'])
Submisssion_de = pd.DataFrame()
Submisssion_de['Text'] = de_test['text']
Submisssion_de['Tag'] = y_pred_de
Submisssion_de.to_csv('predictions.csv', index = None)

# **Multiclass**

**DEV set**

In [17]:

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000)),
    ('clf', LogisticRegression(max_iter=1000))
])

pipeline.fit(merge_train['processed_text'], merge_train['multiclass'])


y_pred_es = pipeline.predict(es_dev['processed_text'])
y_pred_en = pipeline.predict(en_dev['processed_text'])
y_pred_ur = pipeline.predict(ur_dev['processed_text'])
y_pred_de = pipeline.predict(de_dev['processed_text'])


In [18]:

print("Classification Report for Spanish (es):")
print(classification_report(es_dev['multiclass'], y_pred_es))

print("Classification Report for English (en):")
print(classification_report(en_dev['multiclass'], y_pred_en))

print("Classification Report for Urdu (ur):")
print(classification_report(ur_dev['multiclass'], y_pred_ur))

print("Classification Report for German (de):")
print(classification_report(de_dev['multiclass'], y_pred_de))

Classification Report for Spanish (es):
                  precision    recall  f1-score   support

Generalized Hope       0.61      0.62      0.61      1001
        Not Hope       0.71      0.87      0.78      1958
  Realistic Hope       0.54      0.20      0.29       405
Unrealistic Hope       0.52      0.29      0.37       473

        accuracy                           0.66      3837
       macro avg       0.59      0.49      0.51      3837
    weighted avg       0.64      0.66      0.63      3837

Classification Report for English (en):
                  precision    recall  f1-score   support

Generalized Hope       0.55      0.64      0.59       467
        Not Hope       0.71      0.85      0.77       816
  Realistic Hope       0.56      0.14      0.23       196
Unrealistic Hope       0.57      0.27      0.37       171

        accuracy                           0.64      1650
       macro avg       0.60      0.47      0.49      1650
    weighted avg       0.63      0.64      0.

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


**Test Set**

In [19]:
y_pred_es = pipeline.predict(es_test['processed_text'])
y_pred_en = pipeline.predict(en_test['processed_text'])
y_pred_ur = pipeline.predict(ur_test['processed_text'])
y_pred_de = pipeline.predict(de_test['processed_text'])

In [20]:
#submission file

y_pred_es = pd.DataFrame(data=y_pred_es, columns=['final_prediction'])
Submisssion_es = pd.DataFrame()
Submisssion_es['Text'] = es_test['text']
Submisssion_es['Tag'] = y_pred_es
Submisssion_es.to_csv('predictions.csv', index = None)

In [21]:
#submission file

y_pred_en = pd.DataFrame(data=y_pred_en, columns=['final_prediction'])
Submisssion_en = pd.DataFrame()
Submisssion_en['Text'] = en_test['text']
Submisssion_en['Tag'] = y_pred_en
Submisssion_en.to_csv('predictions.csv', index = None)

In [22]:
#submission file

y_pred_ur = pd.DataFrame(data=y_pred_ur, columns=['final_prediction'])
Submisssion_ur = pd.DataFrame()
Submisssion_ur['Text'] = ur_test['text']
Submisssion_ur['Tag'] = y_pred_ur
Submisssion_ur.to_csv('predictions.csv', index = None)

In [23]:
#submission file

y_pred_de = pd.DataFrame(data=y_pred_de, columns=['final_prediction'])
Submisssion_de = pd.DataFrame()
Submisssion_de['Text'] = de_test['text']
Submisssion_de['Tag'] = y_pred_de
Submisssion_de.to_csv('predictions.csv', index = None)